In [2]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\1998a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1998a\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Step 2: Data Loading
data = pd.read_csv("D:/Python Projects/Multimodal Neural Net for House Price Predication/data/austinHousingData.csv")
data.head(2)

,zpid,city,streetAddress,zipcode,description,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,...,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,homeImage
0,111373431,pflugerville,14424 Lake Victor Dr,78660,"14424 Lake Victor Dr, Pflugerville, TX 78660 i...",30.430632,-97.663078,1.98,2,True,...,1,1,1.266667,2.666667,1063,14,3.0,4,2,111373431_ffce26843283d3365c11d81b8e6bdc6f-p_f...
1,120900430,pflugerville,1104 Strickling Dr,78660,Absolutely GORGEOUS 4 Bedroom home with 2 full...,30.432673,-97.661697,1.98,2,True,...,1,1,1.400000,2.666667,1063,14,2.0,4,1,120900430_8255c127be8dcf0a1a18b7563d987088-p_f...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15171 entries, 0 to 15170
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zpid                        15171 non-null  int64  
 1   city                        15171 non-null  object 
 2   streetAddress               15171 non-null  object 
 3   zipcode                     15171 non-null  int64  
 4   description                 15169 non-null  object 
 5   latitude                    15171 non-null  float64
 6   longitude                   15171 non-null  float64
 7   propertyTaxRate             15171 non-null  float64
 8   garageSpaces                15171 non-null  int64  
 9   hasAssociation              15171 non-null  bool   
 10  hasCooling                  15171 non-null  bool   
 11  hasGarage                   15171 non-null  bool   
 12  hasHeating                  15171 non-null  bool   
 13  hasSpa                      151

In [9]:
data[["yearBuilt"]]

,yearBuilt
0,2012
1,2013
2,2018
3,2013
4,2002
...,...
15166,1986
15167,2016
15168,1936
15169,1968


In [10]:
data[["numOfBathrooms"]].value_counts()

numOfBathrooms
2.00              6101
3.00              5280
4.00              1884
1.00               932
5.00               408
6.00               157
2.50               126
0.00               125
7.00                61
3.50                44
8.00                17
1.50                13
4.50                 5
10.00                5
1.75                 3
9.00                 2
5.50                 2
2.75                 2
1.70                 1
6.50                 1
13.00                1
27.00                1
Name: count, dtype: int64

In [11]:
# Step 3: Apply bathroom and bedroom transformation

data.loc[(data['numOfBathrooms'] == 0) & (data['yearBuilt'] > 1989), 'numOfBathrooms'] = 2
data.loc[(data['numOfBathrooms'] == 0) & (data['yearBuilt'] <= 1989), 'numOfBathrooms'] = 1

data.loc[(data['numOfBedrooms'] == 0) & (data['yearBuilt'] > 1989), 'numOfBedrooms'] = 2.5
data.loc[(data['numOfBedrooms'] == 0) & (data['yearBuilt'] <= 1989), 'numOfBedrooms'] = 2

C:\Users\1998a\AppData\Local\Temp\ipykernel_68408\1552460035.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[(data['numOfBedrooms'] == 0) & (data['yearBuilt'] > 1989), 'numOfBedrooms'] = 2.5


In [12]:
data[["numOfBedrooms"]].value_counts()

numOfBedrooms
3.0              7440
4.0              5190
5.0              1171
2.0              1098
6.0               128
1.0                98
8.0                18
2.5                13
7.0                13
10.0                1
20.0                1
Name: count, dtype: int64

In [13]:
data[["homeType"]].value_counts()

homeType             
Single Family            14241
Condo                      470
Townhouse                  174
Multiple Occupancy          96
Vacant Land                 83
Apartment                   37
Residential                 37
Mobile / Manufactured       17
MultiFamily                 10
Other                        6
Name: count, dtype: int64

In [14]:
data[["latestPrice"]]

,latestPrice
0,305000.0
1,295000.0
2,256125.0
3,240000.0
4,239900.0
...,...
15166,330000.0
15167,550000.0
15168,875000.0
15169,420000.0


In [15]:
data[["garageSpaces"]].value_counts()

garageSpaces
0               6826
2               5508
1               1098
3                938
4                581
6                 99
5                 74
8                 17
7                 11
10                 9
12                 5
9                  3
20                 1
22                 1
Name: count, dtype: int64

In [16]:
# step 4: Cap garageSpaces and parkingSpaces
condition = (data['latestPrice'] < 1000000) & (data['homeType'] == "Single Family")
data.loc[condition, 'garageSpaces'] = data.loc[condition, 'garageSpaces'].clip(upper=3)
data.loc[condition, 'parkingSpaces'] = data.loc[condition, 'parkingSpaces'].clip(upper=3)

In [17]:
# steps 5: Drop outliers in lotSizeSqFt and livingAreaSqFt
for col in ['lotSizeSqFt', 'livingAreaSqFt']:
    data = data[data[col] > 0]
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.6 * IQR
    lower = Q1 - 1.6 * IQR
    data = data[(data[col] >= lower) & (data[col] <= upper)]

In [18]:
data[["lotSizeSqFt"]].value_counts().sum()

np.int64(13713)

In [19]:
data[["livingAreaSqFt"]].value_counts().sum()

np.int64(13713)

In [20]:
# Steps 6: Adjust price for inflation (placeholder logic)
def adjust_inflation(price, year):
    inflation_rate = 0.03  # 3% yearly inflation
    years_passed = 2025 - year
    return round(price * ((1 + inflation_rate) ** years_passed), 2)

In [21]:
data['newPrice'] = data.apply(lambda x: adjust_inflation(x['latestPrice'], x['latest_saleyear']), axis=1)

In [22]:
data["latest_saleyear"].value_counts()

latest_saleyear
2020    4895
2019    4783
2018    3960
2021      75
Name: count, dtype: int64

In [23]:
# Step 7: Drop invalid latitude values
data = data[data['latitude'] > 30.12]

In [24]:
# step 8: Drop out-of-Austin zipcodes (placeholder - use actual zip list)
valid_zipcodes = [
    73301, 73344, 78617, 78652, 78653, 78660, 78701, 78702, 78703, 
    78704, 78705, 78717, 78719, 78721, 78722, 78723, 78724, 78725, 
    78726, 78727, 78728, 78729, 78730, 78731, 78732, 78733, 78734, 
    78735, 78736, 78737, 78738, 78739, 78741, 78742, 78744, 78745, 
    78746, 78747, 78748, 78749, 78750, 78751, 78752, 78753, 78754, 
    78756, 78757, 78758, 78759
]  # list of valid zipcodes in Austin

data = data[data['zipcode'].isin(valid_zipcodes)]

In [ ]:
data[[""]]

In [26]:
# Cell 10: Convert feature counts > 0 to binary
feature_cols = [
    'numOfAccessibilityFeatures', 'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
    'numOfWaterfrontFeatures', 'numOfWindowFeatures', 'numOfCommunityFeatures'
]
for col in feature_cols:
    data[col] = (data[col] > 0).astype(int)

In [27]:
data[["streetAddress"]]

,streetAddress
0,14424 Lake Victor Dr
1,1104 Strickling Dr
2,1408 Fort Dessau Rd
3,1025 Strickling Dr
4,15005 Donna Jane Loop
...,...
15165,9013 Marsh Dr
15166,905 Silcantu Dr
15167,1413 Waterloo Shore Ln UNIT 12
15168,2206 S 3rd St


In [ ]:
# Cell 10: Extract address type from streetAddress
def extract_street_type(address):
    types = ['drive', 'lane', 'street', 'cove', 'road', 'avenue', 'trail', 'circle', 'court', 'way', 'dr', 'ln', 'rd', 'st']
    if pd.isna(address): return 'unknown'
    address = address.lower()
    for t in types:
        if t in address:
            return t
    return 'other'

data['streetType'] = data['streetAddress'].apply(extract_street_type)

In [35]:
data[["streetType"]].value_counts()

streetType
other         13112
way             412
avenue           60
road             36
trail            26
cove             25
circle           15
court            14
lane             13
Name: count, dtype: int64

In [33]:
data[["streetAddress", "streetType"]]

,streetAddress,streetType
0,14424 Lake Victor Dr,other
1,1104 Strickling Dr,other
2,1408 Fort Dessau Rd,other
3,1025 Strickling Dr,other
4,15005 Donna Jane Loop,other
...,...,...
15165,9013 Marsh Dr,other
15166,905 Silcantu Dr,other
15167,1413 Waterloo Shore Ln UNIT 12,other
15168,2206 S 3rd St,other


In [7]:
data[["description"]]

,description
0,"14424 Lake Victor Dr, Pflugerville, TX 78660 i..."
1,Absolutely GORGEOUS 4 Bedroom home with 2 full...
2,Under construction - estimated completion in A...
3,Absolutely darling one story home in charming ...
4,Brimming with appeal & warm livability! Sleek ...
...,...
15166,Not for rent!\n\nBeautiful one story home in T...
15167,This contemporary three-story detached condo h...
15168,Original bungalow with a modern addition. Larg...
15169,So many options at this address in the eclecti...


In [ ]:
# Step 11: Clean description text
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text): return ""
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

data['clean_description'] = data['description'].apply(clean_text)

In [ ]:
# Step 12: Geospatial clustering (latitude & longitude)
coords = data[['latitude', 'longitude']].dropna()
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

kmeans = KMeans(n_clusters=5, random_state=0)
data.loc[coords.index, 'geo_cluster'] = kmeans.fit_predict(coords_scaled)

In [ ]:
# Step 13: Map census data using zipcode
# Placeholder mapping
census_data = pd.read_csv("census_data.csv")  # should contain zipcode + columns like crime_rate, income, etc.
df = data.merge(census_data, on='zipcode', how='left')

In [ ]:
# Cell 14: Save cleaned dataset
df.to_csv("cleaned_dataset.csv", index=False)